In [248]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('Movie',), ('Genre',), ('Language',), ('Country',), ('Location',), ('M_Location',), ('M_Country',), ('M_Language',), ('M_Genre',), ('Person',), ('M_Producer',), ('M_Director',), ('M_Cast',)]


# 1

In [52]:
%%time
# Write your sql query below

query = """ SELECT P.Name,M.title, M.year 
FROM Person P, Movie M, M_Director D,Genre G, M_Genre MG 
WHERE MG.GID = G.GID AND M.MID = MG.MID AND G.Name LIKE ('%Comedy%') AND D.MID = MG.MID AND D.PID=P.PID AND M.year % 4 = 0"""
q = pd.read_sql_query(query, conn)    
print(q)


                       Name                              title     year
0              Milap Zaveri                         Mastizaade     2016
1              Danny Leiner  Harold & Kumar Go to White Castle     2004
2            Anurag Kashyap                 Gangs of Wasseypur     2012
3              Frank Coraci        Around the World in 80 Days     2004
4             Griffin Dunne             The Accidental Husband     2008
5               Anurag Basu                             Barfi!     2012
6           Gurinder Chadha                  Bride & Prejudice     2004
7                Mike Judge    Beavis and Butt-Head Do America     1996
8               Abhinay Deo                          Blackmail   I 2018
9          Tarun Mansukhani                            Dostana     2008
10             Shakun Batra                      Kapoor & Sons     2016
11            Aditya Chopra                Rab Ne Bana Di Jodi     2008
12             Rohit Dhawan                            Dishoom  

# 2

In [38]:
%%time
# Write your sql query below

query = """select Name from Person where PID IN(select TRIM(PID) from M_Cast where MID IN (select MID from Movie where title='Anand' and year=1971))"""
q = pd.read_sql_query(query, conn)    
print(q)


                 Name
0    Amitabh Bachchan
1       Rajesh Khanna
2       Sumita Sanyal
3          Ramesh Deo
4           Seema Deo
5      Asit Kumar Sen
6          Dev Kishan
7        Atam Prakash
8       Lalita Kumari
9              Savita
10     Brahm Bhardwaj
11       Gurnam Singh
12       Lalita Pawar
13        Durga Khote
14         Dara Singh
15      Johnny Walker
16          Moolchand
Wall time: 52 ms


# 3

In [42]:
%%time
query = """select Name from Person where (PID IN(select trim(PID) as ID from M_Cast where MID IN(select Distinct(MID) from Movie where year<1970)) AND PID IN (select trim(PID) as ID from M_Cast where MID IN(select Distinct(MID) from Movie where year>1990)))"""
q = pd.read_sql_query(query, conn)    
print(q)

                    Name
0           Rishi Kapoor
1       Amitabh Bachchan
2                 Asrani
3           Zohra Sehgal
4        Parikshat Sahni
5          Rakesh Sharma
6            Sanjay Dutt
7              Ric Young
8                  Yusuf
9         Suhasini Mulay
10           A.K. Hangal
11          Jeremy Child
12          Farida Jalal
13        Waheeda Rehman
14         Rajesh Khanna
15            Ramesh Deo
16             Seema Deo
17        Asit Kumar Sen
18         Lalita Kumari
19        Brahm Bhardwaj
20          Lalita Pawar
21            Dara Singh
22         Johnny Walker
23             Moolchand
24            Saira Banu
25           Prem Chopra
26           Dina Pathak
27        Achala Sachdev
28            Shashikala
29    Mohandas K. Gandhi
..                   ...
303       Gemini Ganesan
304                 Aziz
305          Mohamad Ali
306          Master Amar
307                Gopal
308               Manish
309             Surendra
310            Raj Joshi


# 4

In [28]:
%%time
# Write your sql query below

query = """ SELECT Name, COUNT(MID)
FROM Person P, M_Director D
WHERE P.PID = D.PID
GROUP BY D.PID
HAVING COUNT(MID) >= 10
ORDER BY COUNT(MID) DESC"""
q = pd.read_sql_query(query, conn)    
print(q)


                         Name  COUNT(MID)
0                David Dhawan          39
1                Mahesh Bhatt          35
2                Priyadarshan          30
3             Ram Gopal Varma          30
4                Vikram Bhatt          29
5        Hrishikesh Mukherjee          27
6                 Yash Chopra          21
7             Basu Chatterjee          19
8              Shakti Samanta          19
9                Subhash Ghai          18
10              Shyam Benegal          17
11   Abbas Alibhai Burmawalla          17
12          Rama Rao Tatineni          17
13             Manmohan Desai          16
14                     Gulzar          16
15               Raj N. Sippy          16
16                 Raj Kanwar          15
17           Mahesh Manjrekar          15
18                Indra Kumar          14
19                 Raj Khosla          14
20               Rahul Rawail          14
21          Rajkumar Santoshi          14
22              Rakesh Roshan     

# 5

In [55]:
%%time
# Write your sql query below
query=""" select distinct(cast(SUBSTR(m.year,-4,4) as int)) as year, count(*) as movie_count from Movie m JOIN
                            (select distinct MID from M_Cast  
                                    where MID NOT IN 
                                    (select c.MID from M_Cast c where trim(c.PID) in (
                                        select trim(p.PID) from Person p where p.gender ="Male" or p.gender is null
                                        )or c.PID is null))a on a.MID = m.MID group by m.year"""
q = pd.read_sql_query(query, conn)
print(q)

   year  movie_count
0  1939            1
1  1999            1
2  2000            1
3  2018            1
Wall time: 247 ms


# 5_2

In [54]:
%%time
# Write your sql query below
q = pd.read_sql_query(
                            '''
                            select distinct(cast(SUBSTR(m.year,-4,4) as int)) as year,   
                            cast((i_vw.female_movie_count*100/(count(m.mid))) as varchar) as percentage,count(m.mid) as movies
                            from Movie m 
                            INNER JOIN
                            (
                                select distinct(cast(SUBSTR(m.year,-4,4) as int)) as year, count(*) as female_movie_count from Movie m JOIN
                            (select distinct MID from M_Cast  
                                    where MID NOT IN 
                                    (select c.MID from M_Cast c where trim(c.PID) in (
                                        select trim(p.PID) from Person p where p.gender ="Male" or p.gender is null
                                        )or c.PID is null))a on a.MID = m.MID group by m.year
                            )i_vw on m.year = i_vw.year
                            group by m.year
                            ''', conn)
print(q)

   year percentage  movies
0  1939         50       2
1  1999          1      66
2  2000          1      64
3  2018          1      93
Wall time: 249 ms


# 6

In [69]:
%%time
# Write your sql query below
q=pd.read_sql_query("""select  m.title, max(cast_count)
                            from
                            (
                                select count(*) as cast_count, mid 
                                from M_Cast  group by mid 
                            )c
                            join Movie m on m.mid = c.mid """
            ,conn)
print(q)

           title  max(cast_count)
0  Ocean's Eight              238
Wall time: 94.6 ms


# 7

In [22]:
%%time
# Write your sql query below
q = pd.read_sql_query('''
                                select Decade,st_year, max(movie_counts)
                                from 
                                (
                                    select Decade, year as st_year,count(*) as movie_counts
                                    from
                                    (
                                        select m.year, a.min_year as year, (((m.year-a.min_year)/10)+1) as Decade from Movie m 
                                        join 
                                        (
                                            select min(year) as min_year from Movie
                                        )a 
                                    )b 
                                    group by Decade
                                )c 
                            ''', conn)
print(q)

   Decade st_year  max(movie_counts)
0       8    2008               1012
Wall time: 82.7 ms


# 8

In [162]:
%%time
# Write your sql query below
q=pd.read_sql_query('''
                                select y.*
                                from
                                (
                                     
                                    select distinct trim(p.pid) as Actor_Id,trim(p.name),   
                                    count(distinct m.mid) as Non_Yash_Chopra_Directed_Movies
                                    from Person p 
                                    join M_Cast mc on trim(mc.pid) = p.pid 
                                    join Movie m on m.mid = mc.mid 
                                    join M_Director md on md.mid = m.mid 
                                    join Person p1 on p1.pid = trim(md.pid)
                                    where trim(p1.name) != \'Yash Chopra\'
                                    group by trim(p.pid)
                                )x
                                left join
                                (
                                    select distinct trim(p.pid) as Actor_Id, 
                                    trim(p.name) as Actor_Name,  
                                    trim(p1.name) as Director_Name, 
                                    count(distinct m.mid) as Yash_Chopra_Directed_Movies 
                                    from Person p 
                                    join M_Cast mc on trim(mc.pid) = p.pid 
                                    join Movie m on m.mid = mc.mid 
                                    join M_Director md on md.mid = m.mid 
                                    join Person p1 on p1.pid = trim(md.pid)
                                    where trim(p1.name) = \'Yash Chopra\'
                                    group by trim(p.pid)
                                )y on x.Actor_Id = y.Actor_Id
                                where y.Yash_Chopra_Directed_Movies > x.Non_Yash_Chopra_Directed_Movies
                                
                                
                            ''', conn) 
print(q)

    Actor_Id   Actor_Name Director_Name  Yash_Chopra_Directed_Movies
0  nm0007181  Yash Chopra   Yash Chopra                            2
1  nm1767604  Ashok Verma   Yash Chopra                            2
2  nm3163800        Nazir   Yash Chopra                            2
Wall time: 1.81 s


# 9

In [5]:
%%time
# Write your sql query below
q=pd.read_sql_query("""select Name from Person where ((PID) 
In(SELECT trim(distinct(mc.PID)) from M_Cast mc where (mc.MID)  
IN(select Distinct(mc.MID) from M_cast mc where (mc.PID) 
IN(select Distinct(mc.PID) from M_cast mc where (mc.MID) 
IN(select Distinct(mc.MID) from M_Cast mc where trim(mc.PID) 
IN(select distinct(trim(p.PID)) from Person p where p.Name LIKE '%Shah%Rukh%Khan%')))) and (PID) not IN
(select Distinct(mc.PID) from M_cast mc where (mc.MID) 
IN(select Distinct(mc.MID) from M_Cast mc where trim(mc.PID) 
IN(select distinct(trim(p.PID)) from Person p where p.Name LIKE '%Shah%Rukh%Khan%'))))) """,conn)
print(q)

                           Name
0                  Freida Pinto
1                   Rohan Chand
2                  Damian Young
3               Waris Ahluwalia
4         Caroline Christl Long
5                 Rajeev Pahuja
6             Michelle Santiago
7               Alicia Vikander
8                  Dominic West
9                Walton Goggins
10                    Daniel Wu
11         Kristin Scott Thomas
12                 Derek Jacobi
13           Alexandre Willaume
14                 Tamer Burjaq
15               Adrian Collins
16               Keenan Arrison
17               Andrian Mazive
18                Milton Schorr
19            Hannah John-Kamen
20                 Peter Waison
21                   Samuel Mak
22                     Sky Yang
23                  Civic Chung
24                  Josef Altin
25          Billy Postlethwaite
26       Roger Jean Nsengiyumva
27               Jaime Winstone
28               Michael Obiora
29                Shekhar Varma
...     